In [ ]:
# Import required libraries and dependencies
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
#import dependency for SQLite database
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("../Resources/perfume.db")

# Create a cursor object to execute SQL queries
cur = conn.cursor()

In [ ]:
# Fetch all rows from the 'Review_Included' table
cur.execute("SELECT * FROM Reviews_Included")

# Fetch all rows from the 'Review_Included' table
rows = cur.fetchall()

# Close the cursor and connection
cur.close()
conn.close()

In [ ]:
# Convert fetched data into a pandas DataFrame
# Get column names
columns = [col[0] for col in cur.description]
perfume_df = pd.DataFrame(rows, columns=columns)

# Display the first few rows of the DataFrame
perfume_df.head()

In [ ]:
# Remove unnecessary columns (at this point we have decided to use the main accords in lieu of the top, middle, and base notes for accuracy)
columns_to_remove = ['image', 'for_gender', 'description', 'gender_vote', 'price value', 'top notes', 'middle notes', 'base notes']

# Remove the specified columns
perfume_df = perfume_df.drop(columns=columns_to_remove)

# Display the DataFrame after dropping the specified columns
perfume_df.head()

In [ ]:
# Define the threshold for removing rows based on rating
rating_threshold = 3.59

# Filter the DataFrame to exclude rows with ratings at or below the threshold
perfume_df = perfume_df[perfume_df['rating'] > rating_threshold]

# Display the DataFrame after removing rows
print(perfume_df)

In [ ]:
# Define the threshold for removing rows based on number of votes
votes_threshold = 100

# Filter the DataFrame to exclude rows with number of votes less than the threshold
perfume_df = perfume_df[perfume_df['number_votes'] >= votes_threshold]

# Display the DataFrame after removing rows
print(perfume_df)

In [ ]:
# Rename the DataFrame to reflect its cleaned state
# Make a copy to preserve the original DataFrame
cleaned_perfume_df = perfume_df.copy()
cleaned_perfume_df

In [ ]:
print(cleaned_perfume_df.columns)

In [ ]:
#Working to create a new entry for each scent creating multiple entries for each perfume based on the number of scents in it
# Parse the string representations of dictionaries in 'main accords' into separate rows
accords_df = cleaned_perfume_df['main accords'].apply(lambda x: pd.Series(eval(x)).items())

# Stack the resulting Series to create multiple rows for each perfume
accords_df = accords_df.apply(pd.Series).stack().reset_index(level=1, drop=True).reset_index()

# Rename the columns
accords_df.columns = ['index', 'scent', 'scent_strength']

# Merge accords_df with the original DataFrame
cleaned_perfume_df = cleaned_perfume_df.merge(accords_df, left_index=True, right_on='index')

# Drop unnecessary columns
cleaned_perfume_df.drop(['main accords', 'index'], axis=1, inplace=True)

# Display the cleaned DataFrame
cleaned_perfume_df.head()

In [ ]:

# Create multiple entries for each perfume based on the number of scents it has from the 'main accords' column
perfume_df = cleaned_perfume_df.loc[cleaned_perfume_df.index.repeat(accords_df.shape[1])]
perfume_df.head()

In [ ]:
# Reset index of perfume_df to ensure uniqueness
perfume_df.reset_index(drop=True, inplace=True)
perfume_df.head()

In [ ]:
# Reset index of accords_df to ensure uniqueness
accords_df.reset_index(drop=True, inplace=True)
accords_df.head()

In [ ]:
# Concatenate accords_df with perfume_df
cleaned_perfume_df = pd.concat([perfume_df, accords_df], axis=1)
cleaned_perfume_df.head()

In [ ]:
# Create a new column for scent strength
cleaned_perfume_df['scent_strength'] = cleaned_perfume_df['rating'] * cleaned_perfume_df.iloc[:, 7:]
cleaned_perfume_df.head()

In [ ]:
# Parse the string representations of dictionaries in 'main accords' into separate columns
accords_df = cleaned_perfume_df['main accords'].apply(lambda x: pd.Series(eval(x)))

# Create multiple entries for each perfume based on the number of scents it has from the 'main accords' column
perfume_df = cleaned_perfume_df.loc[cleaned_perfume_df.index.repeat(accords_df.shape[1])]

# Reset index of perfume_df to ensure uniqueness
perfume_df.reset_index(drop=True, inplace=True)

# Reset index of accords_df to ensure uniqueness
accords_df.reset_index(drop=True, inplace=True)

# Concatenate accords_df with perfume_df
cleaned_perfume_df = pd.concat([perfume_df, accords_df], axis=1)

# Create a new column for scent strength
cleaned_perfume_df['scent_strength'] = cleaned_perfume_df['rating'] * cleaned_perfume_df.iloc[:, 7:]

# Display the cleaned DataFrame
cleaned_perfume_df.head()

In [ ]:
# Parse the string representations of dictionaries in 'main accords' into separate columns
accords_df = cleaned_perfume_df['main accords'].apply(lambda x: pd.Series(eval(x)))

# Create multiple entries for each perfume based on the number of scents it has
perfume_df = cleaned_perfume_df.loc[cleaned_perfume_df.index.repeat(accords_df.shape[1])]

# Reset index to align with the duplicated rows
accords_df.reset_index(drop=True, inplace=True)

# Concatenate accords_df with perfume_df
cleaned_perfume_df = pd.concat([perfume_df, accords_df], axis=1)

# Create a new column for scent strength
cleaned_perfume_df['scent_strength'] = cleaned_perfume_df['rating'] * cleaned_perfume_df.iloc[:, 7:]

# Drop unnecessary columns
cleaned_perfume_df.drop(['main accords', 'rating', 'number_votes'], axis=1, inplace=True)

# Handle missing values, if any
cleaned_perfume_df.dropna(inplace=True)

# Display the cleaned DataFrame
cleaned_perfume_df.head()

In [ ]:
# Import specific dependencies for this block
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Feature scaling (Min-Max scaling)
scaler = MinMaxScaler()
accords_columns = [col for col in cleaned_perfume_df.columns if col not in ['rating', 'number_votes']]
accords_scaled = scaler.fit_transform(cleaned_perfume_df[accords_columns])
cleaned_perfume_df[accords_columns] = accords_scaled

# Split the data into training and testing sets
#These are the features
X = cleaned_perfume_df.drop(['rating', 'number_votes'], axis=1)

# Target variable
y = cleaned_perfume_df['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the cleaned DataFrame
cleaned_perfume_df.head()

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the database

# Review the scaled data

In [ ]:
# Create a DataFrame with the scaled data

# Copy the names from the original data

# Set the identifier column as index

# Display sample data

Find the best value for k using the original data

In [ ]:
# Create a list with the number of k-values from 1 to 11

In [ ]:
# Create an empty list to store the inertia values

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `market_scaled`
# 3. Append the model.inertia_ to the inertia list

In [ ]:
# Create a dictionary with the data to plot the Elbow curve

# Create a DataFrame with the data to plot the Elbow curve

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k using hvplot

# Show the line chart

The best value for k is:

Cluster scent notes with K-means using the original data

In [ ]:
# Initialize the K-Means model using the best value for k


In [ ]:
# Fit the K-Means model using the scaled data

In [ ]:
# Predict the clusters to group the notes using the scaled data

# Print the resulting array of cluster values.

In [ ]:
# Create a copy of the DataFrame

In [ ]:
# Add a new column to the DataFrame with the predicted clusters

# Display sample data

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the note name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

# Show scatterplot

Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.

# View the first five rows of the DataFrame.

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.

In [ ]:
# Retrieve the explained variance ratios

# Calculate the total explained variance

# Print the total explained variance

What is the total explained variance of the # principal components?

In [ ]:
# Creating a DataFrame with the PCA data

# Copy the note names from the original data

# Set the name column as index

# Display sample data

Find the best value for k using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11

In [ ]:
# Create an empty list to store the inertia values

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_pca`
# 3. Append the model.inertia_ to the inertia list

In [ ]:
# Create a dictionary with the data to plot the Elbow curve

# Create a DataFrame with the data to plot the Elbow curve

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.

The best value for k when using the PCA data is:

Does the best k value change between the PCA Data and the Original Data?

Cluster Notes with K-means using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k

In [ ]:
# Fit the K-Means model using the PCA data

In [ ]:
# Predict the clusters to group the notes using the PCA data

# Print the resulting array of cluster values.

In [ ]:
# Create a copy of the DataFrame with the PCA data

# Add a new column to the DataFrame with the predicted clusters

# Display sample data

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the note name in the `hover_cols` parameter to identify 
# the note represented by each data point.

# Show the scatter plot

Visualize and Compare the Results

In [ ]:
# Composite plot to contrast the Elbow curves
# Arrange plots side by side for comparison

# Show the composite plot

In [ ]:
# Composite plot to contrast the clusters
# Arrange plots side by side for comparison

# Show the scatter plot comparison